<a href="https://colab.research.google.com/github/boysbytes/colab-chatbot/blob/main/colab_rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [ ]:
!pip install ollama
import os
from google.colab import drive
import ollama

# Mount Google Drive
drive.mount('/content/drive')

ollama_models_path = '/content/drive/MyDrive/ollama_models'

if not os.path.exists(ollama_models_path):
    os.makedirs(ollama_models_path)
    print(f"Created directory: {ollama_models_path}")
else:
    print(f"Directory already exists: {ollama_models_path}")

!curl -fsSL https://ollama.com/install.sh | sh
!pip install -U langchain chromadb sentence-transformers gradio pypdf langchain-community

### Set up Ollama and model directory

In [ ]:
import threading
import subprocess
import time

# Start Ollama in a separate process
def run_ollama_serve():
    subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

running = subprocess.run(["pgrep", "-a", "ollama"], capture_output=True, text=True)
print("Ollama Status:", running.stdout if running.stdout else "Ollama is NOT running!")

!rm -rf ~/.ollama/models
!ln -s /content/drive/MyDrive/ollama_models ~/.ollama/models


### Download the model to Google Drive

In [ ]:
model_info_path = '/content/drive/MyDrive/ollama_models/manifests/registry.ollama.ai/library/smollm2/latest'

if not os.path.exists(model_info_path):
    !ollama pull smollm2
else:
    print("Model already exists. Skipping download.")

### Implement RAG

1. Create a `rag_data` folder in your Google Drive.
2. Copy your notes (in PDF, TXT, DOCX) into the folder.
3. Run the code cell below.

  > This part may take a while to complete.

In [ ]:
# RAG implementation

from langchain.document_loaders import TextLoader, PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA

# 1. Load Documents (Support for multiple file types)
data_folder = '/content/drive/MyDrive/rag_data'
documents = []

for filename in os.listdir(data_folder):
    file_path = os.path.join(data_folder, filename)
    try:
        if filename.endswith('.txt'):
            loader = TextLoader(file_path)
        elif filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx') or filename.endswith('.csv'):
            loader = UnstructuredFileLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    except Exception as e:
        print(f"Error loading {filename}: {e}")

# 2. Split Documents (Dynamic chunk size based on document type)
chunk_size = 500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# 3. Create Embeddings and Vector Store
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
db = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")
# db.persist()

### Refine your prompt template

In [ ]:
PROMPT_TEMPLATE = """You are a helpful AI tutor for a 14-year-old student. Use the context provided to answer the student's questions clearly and concisely.

If the answer to the question is not found in the context, say "I don't know." Avoid making up information and avoid using your prior knowledge.

Maintain context across multiple turns of conversation.

Context:
{context}

Question: {question}

Answer:
"""

print("RAG setup complete.")

### Define the chatbot function

In [ ]:
def chatbot_response(user_message, chat_history):
    # Retrieve relevant documents with reranking logic
    retriever = db.as_retriever(search_kwargs={"k": 3})  # Retrieve top 3 relevant chunks

    try:
        relevant_documents = retriever.get_relevant_documents(user_message)
        # Rerank results based on relevance score (if available)
        relevant_documents.sort(key=lambda doc: doc.metadata.get('relevance_score', 0), reverse=True)

        # Format the context from top-ranked documents
        context = "\n".join([doc.page_content for doc in relevant_documents])

        # Include chat history in the prompt for multi-turn conversation support
        history_context = "\n".join([f"{role}: {content}" for role, content in chat_history])
        prompt = PROMPT_TEMPLATE.format(context=context, question=user_message) + f"\n\nConversation History:\n{history_context}"

        # Generate response using Qwen2.5 model via Ollama
        # response = ollama.chat(model="qwen2.5:0.5b", messages=[{"role": "user", "content": prompt}])
        response = ollama.chat(model="smollm2", messages=[{"role": "user", "content": prompt}])
        bot_message = response["message"]["content"]
    except Exception as e:
        bot_message = f"Error: {e}"

    # Update chat history and return it along with the bot's response
    chat_history.append(("User", user_message))
    chat_history.append(("Chatbot", bot_message))
    return chat_history, chat_history

### Define the chatbot interface

In [ ]:
import gradio as gr

with gr.Blocks() as chatui:
    gr.Markdown("# RAG-Enabled Chatbot")

    # Improved Chatbot Display
    chatbot_display = gr.Chatbot(
        height=300,  # Adjust height as needed
    )

    # User Input with Styling
    user_input_box = gr.Textbox(
        label="Your Message",
        placeholder="Ask me anything!",
        container=True,
        scale=7,
    )

    # Submit Button with Styling
    send_button = gr.Button(
        "Send",
        variant="primary",  # More visually prominent
        scale=1,
    )

    # Clear Button
    clear_button = gr.ClearButton([user_input_box, chatbot_display], value="Clear")

    # State for Conversation History
    chat_history = gr.State([])

    # Define Functionality
    send_button.click(
        chatbot_response,
        inputs=[user_input_box, chat_history],
        outputs=[chatbot_display, chat_history],
    )



# Launch the chatbot UI with sharing enabled
chatui.launch(share=True)
